In [11]:
import numpy as np
import pandas as pd
import joblib
from sklearn.feature_selection import RFECV
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score, recall_score
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

In [12]:
train_data = pd.read_csv('../../data/raw/train.csv',  encoding= 'unicode_escape')
y_train = train_data["Unusual"]                      #defining the labels
X_train = train_data.drop(["Unusual"], axis=1)
test_data = pd.read_csv('../../data/raw/test.csv',  encoding= 'unicode_escape')
y_test = test_data["Unusual"]                      #defining the labels
X_test = test_data.drop(["Unusual"], axis=1)

In [14]:
# Define the model pipeline
model_pipeline = Pipeline([
    ('preprocessing', joblib.load('../pipelines/PreprocessingPipeline.joblib')),
    ('feature_selection', RFECV(estimator=XGBClassifier())),
    ('xgboost', XGBClassifier()),    
])

# Evaluate the performance of the pipeline using cross-validation
scores = cross_val_score(model_pipeline, X_train, y_train, cv=5)
# Fit the model pipeline to the training data
model_pipeline.fit(X_train, y_train)

# Get the selected features
selected_features = model_pipeline.named_steps['feature_selection'].get_support() 

predict = model_pipeline.predict(X_test)
print(recall_score(predict,y_test))

0.9982758620689656


In [15]:
scores

array([0.98800077, 0.9903232 , 0.99186992, 0.99225707, 0.9912892 ])

In [16]:
selected_features

array([False,  True,  True, False, False, False, False,  True,  True,
        True,  True, False, False, False, False, False,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False])